In [6]:
def read_file(input_file):
    """Read content from a file."""
    try:
        with open(input_file, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{input_file}' was not found.")

def write_file(output_file, content):
    """Write content to a file."""
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(content)
    print(f"Content successfully saved to {output_file}")


In [7]:
def load_dictionary(file_path):
    """Load words from a dictionary file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return set(line.strip() for line in file)


In [8]:
from collections import Counter

def word_length_similarity(misspelt_word, candidate_word):
    """Binary score: 1.0 if lengths match, 0.0 otherwise."""
    return 1.0 if len(misspelt_word) == len(candidate_word) else 0.0

def character_frequency_similarity(misspelt_word, candidate_word):
    """Compute similarity based on character frequency."""
    misspelt_counter = Counter(misspelt_word)
    candidate_counter = Counter(candidate_word)
    matching_count = sum(min(misspelt_counter[char], candidate_counter[char]) for char in misspelt_counter)
    return matching_count / len(misspelt_word)

def position_similarity(misspelt_word, candidate_word):
    """Count matching characters at the same positions."""
    match_count = sum(1 for m_char, c_char in zip(misspelt_word, candidate_word) if m_char == c_char)
    return match_count / min(len(misspelt_word), len(candidate_word))

def first_letter_similarity(misspelt_word, candidate_word):
    """Check if the first letters of both words match."""
    return 1.0 if misspelt_word[0] == candidate_word[0] else 0.0


In [9]:
def levenshtein_distance(word1, word2):
    """Compute the Levenshtein distance between two words."""
    len_word1, len_word2 = len(word1), len(word2)
    matrix = [[0] * (len_word2 + 1) for _ in range(len_word1 + 1)]

    for i in range(len_word1 + 1):
        matrix[i][0] = i
    for j in range(len_word2 + 1):
        matrix[0][j] = j

    for i in range(1, len_word1 + 1):
        for j in range(1, len_word2 + 1):
            cost = 0 if word1[i-1] == word2[j-1] else 1
            matrix[i][j] = min(
                matrix[i-1][j] + 1,  # Deletion
                matrix[i][j-1] + 1,  # Insertion
                matrix[i-1][j-1] + cost  # Substitution
            )

    return matrix[len_word1][len_word2]

def distance_similarity(misspelt_word, candidate_word):
    """Levenshtein distance normalized by maximum possible distance."""
    max_distance = max(len(misspelt_word), len(candidate_word))
    lev_distance = levenshtein_distance(misspelt_word, candidate_word)
    return 1 - (lev_distance / max_distance)


In [10]:
def calculate_similarity_score(misspelt_word, candidate_word):
    """Calculate weighted similarity score between two words."""
    # Weights
    weights = {
        "length": 0.2,
        "frequency": 0.2,
        "position": 0.2,
        "first_letter": 0.2,
        "distance": 0.2
    }

    # Compute heuristic scores
    scores = {
        "length": word_length_similarity(misspelt_word, candidate_word),
        "frequency": character_frequency_similarity(misspelt_word, candidate_word),
        "position": position_similarity(misspelt_word, candidate_word),
        "first_letter": first_letter_similarity(misspelt_word, candidate_word),
        "distance": distance_similarity(misspelt_word, candidate_word)
    }

    # Weighted sum of the scores
    return sum(weights[key] * scores[key] for key in weights)


In [11]:
def suggest_corrections(misspelt_word, dictionary, threshold=0.5):
    """Suggest corrections for a misspelt word."""
    scored_candidates = [
        (candidate, calculate_similarity_score(misspelt_word, candidate))
        for candidate in dictionary
    ]
    # Filter candidates above the threshold
    scored_candidates = [(word, score) for word, score in scored_candidates if score > threshold]
    # Sort by similarity score in descending order
    return sorted(scored_candidates, key=lambda x: x[1], reverse=True)


In [33]:
def process_sentence(sentence, dictionary):
    """
    Process a sentence to find corrections for each word and generate the corrected sentence.

    Parameters:
    - sentence: str, the input sentence to be corrected.
    - dictionary: set, a set of valid Tamil words.

    Returns:
    - corrections: dict, mapping of each word to its suggestions or status.
    - corrected_sentence: str, the sentence after applying corrections.
    """
    words = sentence.split()
    corrections = {}
    corrected_words = []

    for word in words:
        suggestions = suggest_corrections(word, dictionary)
        if not suggestions:
            corrections[word] = "No suggestions"
            corrected_words.append(word)
        elif suggestions[0][1] == 1.0:  # Check for a perfect match
            corrections[word] = "Correct"
            corrected_words.append(word)
        else:
            corrections[word] = [s[0] for s in suggestions]
            corrected_words.append(suggestions[0][0])  # Pick the best suggestion

    corrected_sentence = " ".join(corrected_words)
    return corrections, corrected_sentence


In [43]:
# Example usage
dictionary_file = '/content/tamil_words.txt'  # Replace with your Tamil words file
sentence = 'நான் பூங்க பார்ப்பேன்'  # Replace with your Tamil sentence

# Load dictionary
dictionary = load_dictionary(dictionary_file)

# Process the sentence
corrections, corrected_sentence = process_sentence(sentence, dictionary)

# Print corrections for the sentence
for word, suggestion in corrections.items():
    if suggestion == "Correct":
        print(f"'{word}': Correct")
    elif suggestion == "No suggestions":
        print(f"'{word}': No suggestions found")
    else:
        print(f"'{word}': Suggestions: {', '.join(suggestion)}")

# Print the corrected sentence
print("Corrected Sentence:", corrected_sentence)


'நான்': Correct
'பூங்க': Suggestions: பூங்கா, பூங்கு, பரக்க, பூங்கணை, பூங்கோரை, பூங்கலன், பூங்கழல், பூங்கொடி, பூங்காவி, பூங்காடு, பூங்காலி, பூங்கீரை, பூகம், பங்கி, பூங்கதிர், பூங்குழல், பூங்கஞ்சா, பூங்கருவி, பூங்குடம், பூங்குவளை, பூங்கற்று, பூங்காரம், பூங்காற்று, பூங்கோயில், பூங்கொத்தி, பூங்கொல்லை, பூங்குஞ்சு, பூங்காவனம், பூங்கொத்து, பூங்கரும்பு, பூங்குறுணல், பூரம், பூவல், பூவன், பூதன், பூதகி, பூளம், பூசம், பூடன், பூதம், பூடம், பூயம், பூபன், பூதர், பூமன், பூணல், மூங்க, பைங்கூழ், பைங்கூன், புங்கு, பிங்கி, பிங்கள, பரங்கி, பெங்கு, பொங்கு, போங்கு, பாங்கா, படங்கு, பதங்கு, பூட்கை, பறங்கி, பாங்கி, பாங்கு, பனங்கை, பனங்கூடல், பஸ்கி, பூமடை, பலூத், பூவணி, பரப்ப, பூமழை, பால்2, பூசணி, பூபதி, பபூன், பூரணை, பூலதை, பூவணை, பூரணி, பட்கை, பூசனை, பக்கு, பலாங், பூசனி, பக்கி, பூசலை, பிங்கலை, பழங்கறி, பாங்கன், புங்கவி, போங்கம், பனங்களி, புங்கம், பைங்கண், பூக்கம், பழங்கள், பனங்கனி, பிங்கம், பொங்கடி, பூக்கடை, புங்கன், பிங்களை, பழங்கண், படங்கன், பொங்கழி, பதங்கன், பதங்கம், பழங்கதை, படங்கம், பொங்கர், பலங்கனி, பொங

In [103]:
# Example usage
dictionary_file = '/content/tamil_words.txt'  # Replace with your Tamil words file
sentence = 'நான் நேத்து பாலை குடித்தான். அவனுக்கு புத்தகம் புடிச்சு. அவள் பாடம் நன்றாகப் படித்தார். அவர்கள் கல் அடிக்கிறான். கடைசி நேரத்தில், வேலை முடிக்கிறேன்.'  # Replace with your Tamil sentence

# Load dictionary
dictionary = load_dictionary(dictionary_file)

# Process the sentence
corrections, corrected_sentence = process_sentence(sentence, dictionary)

# Print corrections for the sentence
for word, suggestion in corrections.items():
    if suggestion == "Correct":
        print(f"'{word}': Correct")
    elif suggestion == "No suggestions":
        print(f"'{word}': No suggestions found")
    else:
        print(f"'{word}': Suggestions: {', '.join(suggestion)}")

# Print the corrected sentence
print("Corrected Sentence:", corrected_sentence)


'நான்': Correct
'நேத்து': Suggestions: நெத்து, நீத்து, நேற்று, நாத்தி, நேக்கு, நொய்து, நடத்தை, நித்தை, நேர்பு, நொந்து, நேர்வு, நீந்து, நிந்து, நரதுதி, நுந்தை, கேத்து, சேத்து, நொங்கு, நூற்பு, நென்பு, நாட்பு, நெய்வு, நலங்கு, நோன்பு, நுள்ளு, நாட்டு, நாக்கு, நான்கு, நரம்பு, நொட்டு, நொக்கு, நெக்கு, நெம்பு, நுக்கு, நீச்சு, நோதகவு, நரல்வு, நாங்கு, நாளிது, நோம்பு, நூங்கு, நகழ்வு, நீப்பு, நோக்கு, நூக்கு, நெய்தை, நின்று, நெட்டு, நெற்று, நால்கு, நொப்பு, நாமாது, நெஞ்சு, நாப்பு, நாற்று, நிந்தை, நேற்றி, நடப்பு, நாம்பு, நயப்பு, நலப்பு, நான்று, நாஸ்தி, நாந்தி, நொவ்வு, நொய்வு, நேர்மை, நுங்கு, நெடிது, நெல்லு, நூப்பு, நேத்திராம்பு, நேத்திரநரம்பு, நேர்த்தரவு, நடத்துநர், நரத்துவம், நேத்திரம், நொதுமி, நிதனம், நேசகன், நிதகம், நாவேறு, நோர்வே, நூதனம், நேமனம், சேந்து, வேந்து, நிருதி, நாதார், நுதலணி, நும்பி, நுவ்வை, நதிபதி, நருமதை, நுண்மை, நாதான், நற்கதி, நைருதி, நகநுதி, நிதலம், நூதனன், நுங்கை, நிகுதி, நவதளம், நேமகம், நேர்த்தி, நேத்திரயோனி, நவத்துவாரம், நேத்திரரோகம், நேர்தப்புதல், நேர்முகத்தேர்வு, நேர்த்தல், நேத